In [0]:
import numpy as np
import pandas as pd
from google.colab import drive
import tensorflow as tf
import zipfile
import nltk
import os

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [9]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data = pd.read_csv('gdrive/My Drive/ASAG Work/All Data/Mohler_till_POS.csv')

In [0]:
import csv
#Open file with "w"(write), the file will create automatically.
file = open("gdrive/My Drive/ASAG Work/All Data/moh_pairs.txt", "w")
#Open CSV file to read CSV, note: reading and write file should be under "with"
with open('gdrive/My Drive/ASAG Work/All Data/Mohler_till_POS.csv') as csvFile:
    #Read CSV
    readCsv = csv.reader(csvFile)
    for row in readCsv:
        #Get Values and manupilate in the file.write
        Id = row[1]
        Id1 = row[2]
        #Write CSV you need format it to string if the value is an int
        file.write(Id+'\t'+Id1+'\n')
#You Must Close the FIle after writing it.
file.close()

In [0]:
data.iloc[:,[0,1,2]].to_csv('gdrive/My Drive/ASAG Work/All Data/moh_pairs.tsv',sep='\t',index=None)

## SemEval data merge

In [0]:
dname = 'SciEntsBank' # 'Beetle'

In [0]:
q = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval/"+dname+"_All_Questions.csv",sep=',')

In [0]:
r = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval/"+dname+"_All_Reference_Answers.csv",sep=',')

In [0]:
s = pd.read_csv("gdrive/My Drive/ASAG Work/All Data/SemEval/"+dname+"_All_Student_Answers.csv",sep=',')

In [20]:
q.shape, r.shape, s.shape

((1053, 3), (1053, 5), (32652, 5))

In [21]:
q.columns, r.columns, s.columns

(Index(['qid', 'question', 'train/test'], dtype='object'),
 Index(['qid', 'ref_id', 'ref_answer', 'category', 'train/test'], dtype='object'),
 Index(['qid', 'ref_id', 'stu_answer', 'accuracy', 'train/test'], dtype='object'))

In [28]:
r.head()

,qid,ref_id,ref_answer,category,train/test
0,EM_45b,EM_45b-a1,"The water was evaporated, leaving the salt.",NaN,test-unseen-answers_2way
1,EM_45c,EM_45c-a1,The crystals were square with Xs on the surface.,NaN,test-unseen-answers_2way
2,EM_16b,EM_16b-a1,Topaz would be the hardest of the 4 minerals b...,NaN,test-unseen-answers_2way
3,EM_21a,EM_21a-a1,Rub the minerals together and see which one sc...,NaN,test-unseen-answers_2way
4,EM_21b,EM_21b-a1,The harder mineral will leave a scratch on the...,NaN,test-unseen-answers_2way


In [0]:
for i in range(len(s)):
    if s['ref_id'][i] in [np.nan,np.NaN,np.NAN]:
        s['ref_id'][i] = r[r['qid']==s['qid'][i]]['ref_id'].values[0]

In [23]:
l = []
for i in range(len(s)):
    if i%1000 == 0 and i!= 0:
        print(i)
    qid = s.iloc[i,0]
    quest = q[q['qid']==qid].values[0,1]
    ref_id = s.iloc[i,1]
    ref_ans = r[r['ref_id']==ref_id]['ref_answer'].values[0]
#     cat = r[(r['ref_id']==ref_id)&(r['qid']==qid)]['category'].values[0]
    stu_ans = s.iloc[i,2]
    acc = s.iloc[i,3]
    tr_ts = s.iloc[i,4]
    l.append([qid,ref_id,quest,ref_ans,stu_ans,tr_ts,acc])

IndexError: ignored

In [0]:
d = pd.DataFrame(l,columns=['qid','ref_id','question','ref_answer','stu_answer','train/test','accuracy'])

In [0]:
d.shape

(48249, 7)

In [0]:
d.to_csv("gdrive/My Drive/ASAG Work/All Data/SemEval Full Data.csv",sep=',',index=None)

In [0]:
tr = d[d['train/test'].str.contains('tr')].reset_index(drop=True)
ts = d[d['train/test'].str.contains('test')].reset_index(drop=True)
rl = d[d['train/test'].str.contains('reli')].reset_index(drop=True)

In [0]:
d[(d['train/test'].str.contains('train')) & (d['train/test'].str.contains('2'))].shape

(8910, 7)

In [0]:
p = pd.concat([tr,ts,rl],axis=0)

In [0]:
p.shape

(48249, 7)

In [0]:
p.to_csv("gdrive/My Drive/ASAG Work/All Data/SemEval Full Data.csv",sep=',',index=None)

In [0]:
# assume qsv, rav, av are numpy arrays of dimensions (None,max_q_len,v_len), (None,max_r_len,v_len), (None,max_a_len,v_len)

# InferSent Sentence Embedding

In [0]:
# creating all directories req
try:
  # Copy from drive to local workspace
  ! mkdir ./InferSent
  ! cp -fR gdrive/My\ Drive/ASAG\ Work/InferSent-master/* ./InferSent
  ! mkdir ./InferSent/dataset/GloVe
  ! mkdir ./InferSent/dataset/fastText
except:
  pass

# Download GloVe
! curl -Lo ./InferSent/dataset/GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip ./InferSent/dataset/GloVe/glove.840B.300d.zip -d ./InferSent/dataset/GloVe/

# Download Fasttext
! curl -Lo ./InferSent/dataset/fastText/crawl-300d-2M.vec.zip https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip
! unzip ./InferSent/dataset/fastText/crawl-300d-2M.vec.zip -d ./InferSent/dataset/fastText/

# Download model weights
! curl -Lo ./InferSent/encoder/infersent1.pkl https://s3.amazonaws.com/senteval/infersent/infersent1.pkl
! curl -Lo ./InferSent/encoder/infersent2.pkl https://s3.amazonaws.com/senteval/infersent/infersent2.pkl

mkdir: cannot create directory ‘./InferSent’: File exists
mkdir: cannot create directory ‘./InferSent/dataset/GloVe’: File exists
mkdir: cannot create directory ‘./InferSent/dataset/fastText’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  10.3M      0  0:03:20  0:03:20 --:--:-- 15.0M
Archive:  ./InferSent/dataset/GloVe/glove.840B.300d.zip
  inflating: ./InferSent/dataset/GloVe/glove.840B.300d.txt  
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453M  100 1453M    0     0  28.0M      0  0:00:51  0:00:51 --:--:-- 31.7M
Archive:  ./InferSent/dataset/fastText/crawl-300d-2M.vec.zip
  inflating: ./InferSent/dataset/fastText/crawl-300d-2M.vec  
  

In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
! cp -r gdrive/My\ Drive/ASAG\ Work/InferSent .

^C


In [0]:
from InferSent.models import InferSent
V = 2
MODEL_PATH = './InferSent/encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

In [0]:
W2V_PATH = './InferSent/fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [0]:
infersent.build_vocab(qsv/rav/av, tokenize=True)

In [0]:
embeddings = infersent.encode(qsv/rav/av, tokenize=True)

# Feature Engineering

In [0]:
import pickle

with open("gdrive/My Drive/ASAG/ASAG Work/Mohler1_ques.pickle", 'rb') as pfile:
    questions = pickle.load(pfile)
questions    

{0: 'What is the role of a prototype program in problem solving?\n',
 1: 'What stages in the software life cycle are influenced by the testing stage?\n',
 2: 'What are the main advantages associated with object-oriented programming?\n',
 3: 'Where do C++ programs begin to execute?\n',
 4: 'What is a variable?\n',
 5: 'Where are variables declared in a C++ program?\n',
 6: 'What is typically included in a class definition?\n',
 7: ' What is the difference between a data member and a local variable inside a member function?\n',
 8: ' What is the difference between a constructor and a function?\n',
 9: 'When does C++ create a default constructor?\n',
 10: ' How many constructors can be created for a class?\n',
 11: ' What is the difference between a function prototype and a function definition?\n',
 12: 'What does a function signature include?\n',
 13: ' What is the scope of global variables?\n',
 14: ' How does the compiler handle inline functions?\n',
 15: ' What is the main advantage a

In [0]:
with open("gdrive/My Drive/ASAG/ASAG Work/Mohler1_ans.pickle", 'rb') as pfile:
   answers = pickle.load(pfile)

In [0]:
import pandas as pd

In [0]:
questions[0]

'What is the role of a prototype program in problem solving?\n'

In [0]:
#pd.DataFrame(questions.values(), columns=['ID', 'Question'])
s = pd.Series(questions, name='Questions')
s.index.name = "Id"
s.reset_index()
s

Id
0     What is the role of a prototype program in pro...
1     What stages in the software life cycle are inf...
2     What are the main advantages associated with o...
3             Where do C++ programs begin to execute?\n
4                                 What is a variable?\n
5      Where are variables declared in a C++ program?\n
6     What is typically included in a class definiti...
7      What is the difference between a data member ...
8      What is the difference between a constructor ...
9         When does C++ create a default constructor?\n
10     How many constructors can be created for a cl...
11     What is the difference between a function pro...
12            What does a function signature include?\n
13             What is the scope of global variables?\n
14     How does the compiler handle inline functions?\n
15     What is the main advantage associated with fu...
16    How are overloaded functions differentiated by...
17    When defining a recursive function, wha

In [0]:
df = pd.DataFrame.from_dict(questions,orient='index')
df.columns = ['Question']


,Question
0,What is the role of a prototype program in pro...
1,What stages in the software life cycle are inf...
2,What are the main advantages associated with o...
3,Where do C++ programs begin to execute?\n
4,What is a variable?\n
5,Where are variables declared in a C++ program?\n
6,What is typically included in a class definiti...
7,What is the difference between a data member ...
8,What is the difference between a constructor ...
9,When does C++ create a default constructor?\n


In [0]:
df

,Question,word_count,char_count,avg_word
0,What is the role of a prototype program in pro...,11,60,4.454545
1,What stages in the software life cycle are inf...,13,76,4.846154
2,What are the main advantages associated with o...,9,74,7.222222
3,Where do C++ programs begin to execute?\n,7,40,4.714286
4,What is a variable?\n,4,20,4.000000
5,Where are variables declared in a C++ program?\n,8,47,4.875000
6,What is typically included in a class definiti...,8,50,5.250000
7,What is the difference between a data member ...,17,93,4.750000
8,What is the difference between a constructor ...,11,62,5.100000
9,When does C++ create a default constructor?\n,7,44,5.285714


In [0]:
df['word_count'] = df['Question'].apply(lambda x: len(str(x).split(" ")))
df['char_count'] = df['Question'].str.len()

In [0]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))
df['avg_word'] = df['Question'].apply(lambda x: avg_word(x))